In [2]:
import pandas as pd
import numpy as np
import ast
import decimal
import psycopg2

In [71]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="7798@Voda!!"
)

In [72]:
# Open a cursor to perform database operations
cur = conn.cursor()

In [ ]:
users = []

In [ ]:
def insert_user_info(user):
    try:
        cur.execute("""
        INSERT INTO twitter_users_partitioned 
        (user_id, name, screen_name, date, twitter_join_date, 
        location, description, verified, followers_count, 
        friends_count, listed_count, favourites_count, language)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (user['user_id'], user['name'], user['screen_name'], user['date'], 
          user['twitter_join_date'], user['location'], user['description'], 
          user['verified'], user['followers_count'], user['friends_count'], 
          user['listed_count'], user['favourites_count'], user['language']))
        
        users.append({'user_id': user['user_id'], 'name': user['name'], 
                      'screen_name': user['screen_name'], 'date': user['date'],
                      'twitter_join_date': user['twitter_join_date'], 'location': user['location'], 
                      'description': user['description'], 'verified': user['verified'], 
                      'followers_count': user['followers_count'],'friends_count': user['friends_count'],
                      'listed_count': user['listed_count'], 'favourites_count': user['favourites_count'],
                      'language': user['language']})
        conn.commit()
        cur.close()
        conn.close()

    except Exception as e:
        conn.rollback()
        cur.close()
        conn.close()
        print(e)

In [ ]:
def load_data(file_path):
    # Load the JSON data from file
    with open(file_path, "r") as f:
        for line in f:
            try:
                tweet = json.loads(line)
                user = tweet['user']
                # insert user entry into database
                insert_user_info(user)
                # if there is a retweet, get original user from retweet
                if (tweet['text'].startswith('RT')):
                    original_user = tweet["retweeted_status"]["user"]
                    insert_user_info(original_user)
            except:
                # if there is an error loading the json of the tweet, skip
                continue

In [ ]:
load_data("../corona-out-2")

In [ ]:
load_data("../corona-out-3")

In [ ]:
# Create a pandas DataFrame from the parsed data
df_users = pd.DataFrame(users)

In [29]:
df_users.head()

,user_id,name,screen_name,date,twitter_join_date,location,description,verified,followers_count,friends_count,listed_count,favourites_count,preferred_language
0,804046791348015107,Bi Sex Uau,B_King69,2016-11-30 19:37:48+00:00,2016-11-30 19:37:48+00:00,"Acre, Brasil",se for da minha família já pode voltar daq mesmo,False,89,173,0,5446,NaN
1,2242948745,Thomas Krause,tho1965,2013-12-25 09:13:33+00:00,2013-12-25 09:13:33+00:00,NaN,Sportredakteur @nordkurier 🏃‍♂️🚴‍♂️⚽️,False,173,685,9,2184,NaN
2,908326492718764034,शचीन्द्र पाण्डेय,im_S_pandey,2017-09-14 13:48:06+00:00,2017-09-14 13:48:06+00:00,Amethi Uttar Pradesh,Official Twitter Handel Shachindra Pandey (@im...,False,2362,202,3,30668,NaN
3,2929344220,Ralf Schmitz,RusticusArat,2014-12-18 10:19:26+00:00,2014-12-18 10:19:26+00:00,🇩4790 Provinz,"BWLer,ex Offz,Tw meistens zwischen Tür & Angel...",False,778,733,2,32024,NaN
4,1206650133976408064,Büşra Öztaş,schrodingerk42,2019-12-16 18:59:53+00:00,2019-12-16 18:59:53+00:00,NaN,NaN,False,318,220,0,1974,NaN


In [ ]:
df_users['date'] = pd.to_datetime(df_users['date'])

In [44]:
df_users['twitter_join_date'] = pd.to_datetime(df_users['twitter_join_date'])
df_users.groupby(df_users['twitter_join_date'].dt.year)['twitter_join_date'].count()

twitter_join_date
2006       12
2007      253
2008      920
2009     6933
2010     6688
2011     7672
2012     7400
2013     6457
2014     6102
2015     6018
2016     6865
2017     8138
2018     9573
2019    14966
2020    13908
Name: twitter_join_date, dtype: int64

In [ ]:
df_users.to_csv('data/users.csv', index=False)

Based on the distribution of the rows in the date column, we can create partitions on the data based on its year

Two indexes were recommended to optimize searching for users by their name, number of followers, and verification status. 
A single column index is used for the "name" column, since it is the primary search criterion. A compound index on the "followers_count" and "verified" columns is created to sort and filter records based on the priority of number of followers and verification status. The "followers_count" is specified first in the index definition with a descending order, since sorting in descending order allows for efficient filtering based on the maximum number of followers. These indexing strategies will help speed up query times while minimizing the impact on write performance.

In [58]:
# Create an index on the "name" column
cur.execute("CREATE INDEX name_idx_p ON twitter_users_partitioned (name);")
conn.commit()

# Create a compound index on the "followers_count" and "verified" columns
cur.execute("CREATE INDEX followers_verified_idx_p ON twitter_users_partitioned (followers_count DESC, verified);")
conn.commit()

# Close the database connection
#conn.close()

In [5]:
df_users.head()

,user_id,name,screen_name,date,twitter_join_date,location,description,verified,followers_count,friends_count,listed_count,favourites_count,preferred_language
0,804046791348015107,Bi Sex Uau,B_King69,2020-04-25 12:21:41+00:00,2016-11-30 19:37:48+00:00,"Acre, Brasil",se for da minha família já pode voltar daq mesmo,False,89,173,0,5446,NaN
1,2242948745,Thomas Krause,tho1965,2020-04-25 12:21:41+00:00,2013-12-25 09:13:33+00:00,NaN,Sportredakteur @nordkurier 🏃‍♂️🚴‍♂️⚽️,False,173,685,9,2184,NaN
2,908326492718764034,शचीन्द्र पाण्डेय,im_S_pandey,2020-04-25 12:21:42+00:00,2017-09-14 13:48:06+00:00,Amethi Uttar Pradesh,Official Twitter Handel Shachindra Pandey (@im...,False,2362,202,3,30668,NaN
3,2929344220,Ralf Schmitz,RusticusArat,2020-04-25 12:21:42+00:00,2014-12-18 10:19:26+00:00,🇩4790 Provinz,"BWLer,ex Offz,Tw meistens zwischen Tür & Angel...",False,778,733,2,32024,NaN
4,1206650133976408064,Büşra Öztaş,schrodingerk42,2020-04-25 12:21:42+00:00,2019-12-16 18:59:53+00:00,NaN,NaN,False,318,220,0,1974,NaN


In [73]:
try:
    for index, row in users_df.iterrows():
        cur.execute("""
        INSERT INTO twitter_users_partitioned (user_id, name, screen_name, date, twitter_join_date, location, description, verified, followers_count, friends_count, listed_count, favourites_count, preferred_language)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (row['user_id'], row['name'], row['screen_name'], row['date'], row['twitter_join_date'], row['location'], row['description'], row['verified'], row['followers_count'], row['friends_count'], row['listed_count'], row['favourites_count'], row['preferred_language']))

    conn.commit()
    cur.close()
    conn.close()

except Exception as e:
    conn.rollback()
    cur.close()
    conn.close()
    print(e)


In [23]:
# try:
#     for index, row in users_df.iterrows():
#         cur.execute("""
#             INSERT INTO twitter_users (user_id, name, screen_name, date, twitter_join_date, location, description, verified, followers_count, friends_count, listed_count, favourites_count, preferred_language)
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#             ON CONFLICT (user_id) DO UPDATE
#             SET name = excluded.name,
#                 screen_name = excluded.screen_name,
#                 twitter_join_date = excluded.twitter_join_date,
#                 location = excluded.location,
#                 description = excluded.description,
#                 verified = excluded.verified,
#                 followers_count = excluded.followers_count,
#                 friends_count = excluded.friends_count,
#                 listed_count = excluded.listed_count,
#                 favourites_count = excluded.favourites_count,
#                 preferred_language = excluded.preferred_language;
#         """, (row['user_id'], row['name'], row['screen_name'], row['date'], row['twitter_join_date'], row['location'], row['description'], row['verified'], row['followers_count'], row['friends_count'], row['listed_count'], row['favourites_count'], row['preferred_language']))

# except Exception as e:
#     conn.rollback()
#     cur.close()
#     conn.close()
#     print(e)


In [26]:
pd.read_sql_query('''SELECT EXTRACT(YEAR FROM twitter_join_date) AS year, count(*)
	FROM public.twitter_users group by EXTRACT(YEAR FROM twitter_join_date)
	order by EXTRACT(YEAR FROM twitter_join_date);''', con = conn)

C:\Users\athar\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,count
0,2006.0,12
1,2007.0,253
2,2008.0,917
3,2009.0,6922
4,2010.0,6666
5,2011.0,7664
6,2012.0,7393
7,2013.0,6450
8,2014.0,6087
9,2015.0,6013
